In [1]:
import WebNLG_xmlReader.benchmark_reader as xml_reader
import os.path
import pickle
import pandas as pd
import numpy as np
import re
import tensorflow as tf

In [2]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 

# tokenizer = AutoTokenizer.from_pretrained("t5-base") #"t5-base"

# model = AutoModelForSeq2SeqLM.from_pretrained("t5-base") #TFAutoModelForSeq2SeqLM

from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")


e:\Anaconda\envs\pytorch_test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Anaconda\envs\pytorch_test\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.

In [3]:
from datasets import load_dataset
#dataset = load_dataset("../Data/Cleaned_data/", data_files="Train.csv")
full_dataset = load_dataset("../Data/Cleaned_data/", data_files={"train": "Train.csv", "dev": "Dev.csv", "test": "Test.csv"})

Using custom data configuration Cleaned_data-bcb014efcf526ad6


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 999.12it/s]


Dataset csv downloaded and prepared to C:/Users/Simon/.cache/huggingface/datasets/csv/Cleaned_data-bcb014efcf526ad6/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 999.52it/s]


In [4]:
# zie https://github.com/NielsRogge/Transformers-Tutorials/blob/master/T5/Fine_tuning_Dutch_T5_base_on_CNN_Daily_Mail_for_summarization_(on_TPU_using_HuggingFace_Accelerate).ipynb

def preprocess_data(data):
    max_length = 256
    RDFs = data["input"]
    texts = data["output"]

    model_inputs = tokenizer(RDFs, truncation=True, padding="max_length", return_tensors='pt')

    # specially for seq2seq tokenizer, "Temporarily sets the tokenizer for encoding the targets. Useful for tokenizer associated to sequence-to-sequence models that need a slightly different processing for the labels."
    with tokenizer.as_target_tokenizer():
        target_texts = tokenizer(texts, padding="max_length", truncation=True, return_tensors='pt').input_ids

    #target_texts = tokenizer(texts, truncation=True, padding="max_length").input_ids
    
    #model_inputs['decoder_input_ids'] = np.zeros((len(target_texts), 0))
    model_inputs["labels"] = target_texts
    #print(f'target_texts    {target_texts}')

    # ook nodig

    return model_inputs

In [5]:
encoded_train_ds = full_dataset["train"].map(preprocess_data, batched=True, remove_columns=full_dataset["train"].column_names)  
encoded_dev_ds = full_dataset["dev"].map(preprocess_data, batched=True, remove_columns=full_dataset["dev"].column_names)
encoded_test_ds = full_dataset["test"].map(preprocess_data, batched=True, remove_columns=full_dataset["test"].column_names)

encoded_train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_dev_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


  0%|          | 0/46 [00:00<?, ?ba/s]e:\Anaconda\envs\pytorch_test\lib\site-packages\transformers\tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
100%|██████████| 10/10 [00:01<00:00,  7.96ba/s]


In [6]:
# create smaller dataset chunk
small_train = encoded_train_ds.shard(num_shards = 8, index = 0)
small_val = encoded_dev_ds.shard(num_shards = 8, index = 0)
small_test = encoded_test_ds.shard(num_shards = 8, index = 0)


In [12]:
# Create functions for evaluating the predictions/generations
import datasets
metric = datasets.load_metric("bleu")
def postprocess_text(preds, labels):
    preds = [pred.split() for pred in preds]
    labels = [[label.split()] for label in labels]
    return preds, labels

# def compute_metrics(pred):
#     predictions, labels = pred
#     if isinstance(predictions, tuple):
#         predictions = predictions[0]
#     decoded_preds = tokenizer.batch_decode(predictions,
#     skip_special_tokens=True)

#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels,
#     skip_special_tokens=True)
#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds,
#     decoded_labels)

#     bleu_output = bleu.compute(predictions=decoded_preds,
#     references=decoded_labels)
#     return bleu_output

def compute_metrics(eval_preds):
    #print(f"eval_preds   {eval_preds} \n\n")
    preds, labels = eval_preds #only has 2 values to unpack
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    print(f"preds   {preds} \n\n")
    print(f"labels   {labels} \n\n")

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    #if data_args.ignore_pad_token_for_loss:
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    #print(f"decoded_preds   {decoded_preds} \n\n")
    #print(f"decoded_labels   {decoded_labels} \n\n")

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'results     {result}')

    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


e:\Anaconda\envs\pytorch_test\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    learning_rate=0.001,
    do_eval=True, # will be set to true if evaluation strategy is set
    do_predict=True, #Whether to run predictions on the test set or not.
    evaluation_strategy="steps", 
    eval_steps= 100, # Number of update steps between two evaluations if evaluation_strategy="steps". Will default to the same value as logging_steps if not set.
    save_steps=500, # Number of updates steps before two checkpoint saves if save_strategy="steps".
    max_steps=10, # the total number of training steps to perform
    save_total_limit= 10, # the maximum number of models to keep before deleting the oldest one
    predict_with_generate=True, # Whether to use generate to calculate generative metrics (ROUGE, BLEU).
    generation_num_beams=10,  #The num_beams to use on each evaluation loop when predict_with_generate=True. Will default to the num_beams value of the model configuration
    gradient_checkpointing=True, #
    gradient_accumulation_steps=1, #Number of updates steps to accumulate the gradients for, before performing a backward/update pass
    per_device_train_batch_size=4, #The batch size per GPU/TPU core/CPU for training.
    per_device_eval_batch_size=4, #The batch size per GPU/TPU core/CPU for evaluation.
    optim="adafactor" #The optimizer to use: adamw_hf, adamw_torch, adamw_apex_fused, or adafactor.
    #fp16=True
)

#stepsize vergroten (net als medium post)
#fp16 toepassen

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,  return_tensors="pt")

In [43]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset= small_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer.train()

***** Running training *****
  Num examples = 5734
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 10
  0%|          | 0/10 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
100%|██████████| 10/10 [00:55<00:00,  6.47s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 10/10 [00:55<00:00,  5.53s/it]

{'train_runtime': 55.2744, 'train_samples_per_second': 0.724, 'train_steps_per_second': 0.181, 'train_loss': 2.6537424087524415, 'epoch': 0.01}


TrainOutput(global_step=10, training_loss=2.6537424087524415, metrics={'train_runtime': 55.2744, 'train_samples_per_second': 0.724, 'train_steps_per_second': 0.181, 'train_loss': 2.6537424087524415, 'epoch': 0.01})

In [44]:
trainer.evaluate(small_val) #evaluate on validation set
test_op = trainer.predict(small_test) # predict on test set
print(tokenizer.decode(test_op[0][1],skip_special_tokens=True))

***** Running Evaluation *****
  Num examples = 687
  Batch size = 4


















































































































































































































































































































































































































































































































































































































































































































preds   [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 


labels   [[   71  2335  7532 ...     0     0     0]
 [21512     6     3 ...     0     0     0]
 [   71  4509   568 ...     0     0     0]
 ...
 [ 4603    49  6899 ...     0     0     0]
 [10511  4049     9 ...     0     0     0]
 [   86    20  6494 ...     0     0     0]] 




ZeroDivisionError: float division by zero

### future evals
1. https://github.com/WanzhengZhu/GRUEN
2. https://github.com/ufal/nlgi_eval
3. https://github.com/google-research/bleurt

In [19]:
import torch
torch.is_tensor(small_train['input_ids'][3])

False

In [21]:
encoded_train_ds
torch.is_tensor(encoded_train_ds['input_ids'][3])

100%|██████████| 10/10 [10:23<00:00, 62.39s/it]


False

In [22]:
encoded_train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 45870
})

In [25]:
encoded_train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [27]:
torch.is_tensor(encoded_train_ds['input_ids'][3])

True

In [35]:
tokenizer.decode(small_train[3]['input_ids'])


"['accidentAddress | Alberto’s_Mexican_Food,_1925_West_Holt_Ave.,_east_of_Fairplex_Drive', 'accidentDate | just_after_6:40_p.m.','shootingType | shooting']</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [36]:
tokenizer.decode(small_train[3]['labels'])


'The shooting took place just after 6:40 p.m. at Alberto’s Mexican Food, 1925 West Holt Ave., east of Fairplex Drive, Pomona police Sgt. Steve Congalton said in a written statement.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [7]:
# pip install datasets
import datasets
bleu = datasets.load_metric("bleu")
def postprocess_text(preds, labels):
    preds = [pred.split() for pred in preds]
    labels = [[label.split()] for label in labels]
    return preds, labels

def compute_metrics(pred):
    predictions, labels = pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    #print(f'predictions[0]     {predictions[0]}\n\n')
    #print(f'labels[0]     {labels[0]}\n\n')
     
    decoded_preds = tokenizer.batch_decode(predictions,
    skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels,
    skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
    decoded_labels)

    bleu_output = bleu.compute(predictions=decoded_preds,
    references=decoded_labels)
    return bleu_output


e:\Anaconda\envs\pytorch_test\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = "t5-base-smallDS-story-generation-5Epoch"
model_dir = f"../Results/{model_name}"


training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,
    learning_rate=0.001,
    do_eval=True, # will be set to true if evaluation strategy is set
    do_predict=True, #Whether to run predictions on the test set or not.
    num_train_epochs=5,
    evaluation_strategy="epoch", 
    #eval_steps= 100, # Number of update steps between two evaluations if evaluation_strategy="steps". Will default to the same value as logging_steps if not set.
    save_steps=500, # Number of updates steps before two checkpoint saves if save_strategy="steps".
    #max_steps=10, # the total number of training steps to perform
    save_total_limit= 10, # the maximum number of models to keep before deleting the oldest one
    predict_with_generate=True, # Whether to use generate to calculate generative metrics (ROUGE, BLEU).
    generation_num_beams=10,  #The num_beams to use on each evaluation loop when predict_with_generate=True. Will default to the num_beams value of the model configuration
    gradient_checkpointing=True, #
    gradient_accumulation_steps=1, #Number of updates steps to accumulate the gradients for, before performing a backward/update pass
    per_device_train_batch_size=4, #The batch size per GPU/TPU core/CPU for training.
    per_device_eval_batch_size=4, #The batch size per GPU/TPU core/CPU for evaluation.
    optim="adafactor", #The optimizer to use: adamw_hf, adamw_torch, adamw_apex_fused, or adafactor.
    report_to="tensorboard"

    #fp16=True
)

In [9]:
len(small_train)

5734

In [10]:
def model_init():
    """
    Simple function to ensure that a new model is used for finetuning
    """
    return T5ForConditionalGeneration.from_pretrained("t5-base")

trainer = Seq2SeqTrainer(
model_init=model_init,
#model=model,
args=training_args,
train_dataset=small_train,
eval_dataset=small_val,
compute_metrics=compute_metrics,
tokenizer=tokenizer
)

# max_steps is given, it will override any value given in
# num_train_epochs
# trainer.train(resume_from_checkpoint=True)

loading configuration file config.json from cache at C:\Users\Simon/.cache\huggingface\hub\models--t5-base\snapshots\23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9\config.json
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

In [11]:
# Start TensorBoard before training to monitor it in progress

%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

In [12]:
trainer.train()

loading configuration file config.json from cache at C:\Users\Simon/.cache\huggingface\hub\models--t5-base\snapshots\23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9\config.json
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

{'loss': 0.2088, 'learning_rate': 0.0009302649930264993, 'epoch': 0.35}


Model weights saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-500\tokenizer_config.json
Special tokens file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-500\special_tokens_map.json
Copy vocab file to ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-500\spiece.model
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
 14%|█▍        | 1000/7170 [33:57<3:27:41,  2.02s/it]Saving model checkpoint to ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-1000
Configuration saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-1000\config.json


{'loss': 0.1482, 'learning_rate': 0.0008605299860529985, 'epoch': 0.7}


Model weights saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-1000\special_tokens_map.json
Copy vocab file to ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-1000\spiece.model
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
 20%|██        | 1434/7170 [48:46<2:49:53,  1.78s/it]***** Running Evaluation *****
  Num examples = 687
  Batch size = 4
Trainer is attempting to log a value of "[0.6129943502824858, 0.3656821378340366, 0.22745280820321603, 0.15585135884636717]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.

 20%|██        | 1434/7170 [55:37<2:49:53,  1.78s/it]`us

...

Model weights saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-6500\pytorch_model.bin
tokenizer config file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-6500\tokenizer_config.json
Special tokens file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-6500\special_tokens_map.json
Copy vocab file to ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-6500\spiece.model
Deleting older checkpoint [..\Results\t5-base-smallDS-story-generation-5Epoch\checkpoint-1500] due to args.save_total_limit
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
 98%|█████████▊| 7000/7170 [10:23:12<15:42,  5.55s/it]Saving model checkpoint to ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-7000
Configuration saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-7000\config.json


{'loss': 0.0476, 'learning_rate': 2.3709902370990236e-05, 'epoch': 4.88}


Model weights saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-7000\pytorch_model.bin
tokenizer config file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-7000\tokenizer_config.json
Special tokens file saved in ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-7000\special_tokens_map.json
Copy vocab file to ../Results/t5-base-smallDS-story-generation-5Epoch\checkpoint-7000\spiece.model
Deleting older checkpoint [..\Results\t5-base-smallDS-story-generation-5Epoch\checkpoint-2000] due to args.save_total_limit
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
100%|██████████| 7170/7170 [10:38:53<00:00,  4.85s/it]***** Running Evaluation *****
  Num examples = 687
  Batch size = 4
Trainer is attempting to log a value of "[0.6197767145135566, 0.3922622246104245, 0.26587706759240354, 0.19013750592697962]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboa

{'eval_loss': 0.11812308430671692, 'eval_bleu': 0.1476875940288329, 'eval_precisions': [0.6197767145135566, 0.3922622246104245, 0.26587706759240354, 0.19013750592697962], 'eval_brevity_penalty': 0.44356156958864673, 'eval_length_ratio': 0.5515967273686989, 'eval_translation_length': 6270, 'eval_reference_length': 11367, 'eval_runtime': 1061.7624, 'eval_samples_per_second': 0.647, 'eval_steps_per_second': 0.162, 'epoch': 5.0}
{'train_runtime': 39395.6722, 'train_samples_per_second': 0.728, 'train_steps_per_second': 0.182, 'train_loss': 0.09240128052949573, 'epoch': 5.0}


TrainOutput(global_step=7170, training_loss=0.09240128052949573, metrics={'train_runtime': 39395.6722, 'train_samples_per_second': 0.728, 'train_steps_per_second': 0.182, 'train_loss': 0.09240128052949573, 'epoch': 5.0})

In [13]:
trainer.evaluate(small_val)
test_op = trainer.predict(small_test)
print(tokenizer.decode(test_op[0][1],skip_special_tokens=True))

***** Running Evaluation *****
  Num examples = 687
  Batch size = 4
100%|██████████| 172/172 [17:43<00:00,  6.18s/it]
***** Running Prediction *****
  Num examples = 1136
  Batch size = 4
100%|██████████| 284/284 [34:45<00:00,  7.34s/it]

The men, who are all expected to survive, are between 19 and 19 years old.


In [14]:
# Comparing input RDF to trainer.predict output
for pred_iteration, predictions in enumerate(test_op[0][:5]):
    print(f"input RDF:  {tokenizer.decode(small_test['input_ids'][pred_iteration],skip_special_tokens=True) }\nGeneration:  {tokenizer.decode(predictions,skip_special_tokens=True)}\n\n")
    


input RDF:  ['accidentAddress | southeast_Houston', 'accidentDate | overnight','shootingType | violent_shooting']
Generation:  Police are investigating a violent shooting in southeast Houston overnight.


input RDF:  ['victimAge | 19', 'victimGender | men']
Generation:  The men, who are all expected to survive, are between 19 and 19 years old.


input RDF:  ['victimAge | 39-year-old', 'victimGender | female', 'victimRace | black', 'victimStatus | major_injuries']
Generation:  A 39-year-old female was listed in critical condition with major injuries, police said.


input RDF:  ['victimName | Adrian_Potts']
Generation:  Adrian Potts was shot in the right leg, and he was shot in the right


input RDF:  ["hospitalName | Cook_Children's_Medical_Center", 'takenToHospital | True', 'victimAge | 3-year-olds', 'victimAge_Group | children', 'victimNumber | two', 'victimStatus | serious_condition']
Generation:  The children were taken to Cook Chili's Medical Center in serious condition but are exp

## Save the model/tokenizer/trainer locally

In [28]:
# save the model after fine-tuning so that we can safely test at a later date
#trainer.save_model(f"../Models/{model_name}")
#torch.save(trainer.state_dict(), f"../Models/{model_name}/state_dict")


AttributeError: 'Seq2SeqTrainer' object has no attribute 'state_dict'

In [15]:
#Saves model and tokenizer in one
trainer.save_model(f"../Models/{model_name}") 


Saving model checkpoint to ../Models/t5-base-smallDS-story-generation-5Epoch
Configuration saved in ../Models/t5-base-smallDS-story-generation-5Epoch\config.json
Model weights saved in ../Models/t5-base-smallDS-story-generation-5Epoch\pytorch_model.bin
tokenizer config file saved in ../Models/t5-base-smallDS-story-generation-5Epoch\tokenizer_config.json
Special tokens file saved in ../Models/t5-base-smallDS-story-generation-5Epoch\special_tokens_map.json
Copy vocab file to ../Models/t5-base-smallDS-story-generation-5Epoch\spiece.model


In [29]:
saved_model = T5ForConditionalGeneration.from_pretrained('../Models/t5-base-story-generation/model', local_files_only=True)
saved_tokenizer = AutoTokenizer.from_pretrained('../Models/t5-base-story-generation/model',local_files_only=True)

loading configuration file ../Models/t5-base-story-generation/model\config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_d